**dataframe columns**

In [0]:
airline_df = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .option("samplingRatio", "0.0001")\
    .load("/databricks-datasets/airlines/part-00000")

display(airline_df.limit(5))

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
1987,10,14,3,741,730,912,849,PS,1451,NA,91,79,NA,23,11,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1987,10,15,4,729,730,903,849,PS,1451,NA,94,79,NA,14,-1,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,NO
1987,10,17,6,741,730,918,849,PS,1451,NA,97,79,NA,29,11,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1987,10,18,7,729,730,847,849,PS,1451,NA,78,79,NA,-2,-1,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,NO,NO
1987,10,19,1,749,730,922,849,PS,1451,NA,93,79,NA,33,19,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES


**samplingRatio--Low sampling ratio will load the data faster**

**We have 5 different approach to select columns**
1. "Origin"
2. expr("Origin as xyz")
3. airline_df.Origin
4. column("Origin")
5. col("Origin")

In [0]:
from pyspark.sql.functions import expr, col, column

airline_df.select("Origin", col("Origin"), column("Origin"), airline_df.Origin, expr("Origin"))\
          .show(5)

+------+------+------+------+------+
|Origin|Origin|Origin|Origin|Origin|
+------+------+------+------+------+
|   SAN|   SAN|   SAN|   SAN|   SAN|
|   SAN|   SAN|   SAN|   SAN|   SAN|
|   SAN|   SAN|   SAN|   SAN|   SAN|
|   SAN|   SAN|   SAN|   SAN|   SAN|
|   SAN|   SAN|   SAN|   SAN|   SAN|
+------+------+------+------+------+
only showing top 5 rows


**We want the output as below cell but year, month , day is a seperate column. Also distanceKM should be as Distance*1.6**

In [0]:
%skip

+-------------+---------+----------+----------+
|UniqueCarrier|FlightNum|FlightDate|DistanceKM|
+-------------+---------+----------+----------+
|           PS|     1451|1987-10-14|     715.2|
|           PS|     1451|1987-10-15|     715.2|
|           PS|     1451|1987-10-17|     715.2|
|           PS|     1451|1987-10-18|     715.2|
|           PS|     1451|1987-10-19|     715.2|
+-------------+---------+----------+----------+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import expr
airline_df.select("UniqueCarrier",
                  "Origin",
                  "FlightNum",
                   expr("Year || '-' || Month || '-' || DayofMonth as Flight_Date"),
                   expr("Distance * 1.6 as DistanceKM"))\
            .show(5)

+-------------+------+---------+-----------+----------+
|UniqueCarrier|Origin|FlightNum|Flight_Date|DistanceKM|
+-------------+------+---------+-----------+----------+
|           PS|   SAN|     1451| 1987-10-14|     715.2|
|           PS|   SAN|     1451| 1987-10-15|     715.2|
|           PS|   SAN|     1451| 1987-10-17|     715.2|
|           PS|   SAN|     1451| 1987-10-18|     715.2|
|           PS|   SAN|     1451| 1987-10-19|     715.2|
+-------------+------+---------+-----------+----------+
only showing top 5 rows


**We will create another dataframe having origin and origin airport name**

In [0]:
airport_list = [("SAN", "San Antonio"),
             ("SFO", "San Francisco"),
             ("BUR", "Hollywood Burbank"),
             ("OAK", "Oakland")]
airport_df = spark.createDataFrame(airport_list).toDF("Origin", "OriginAirport_Name")
display(airport_df)

Origin,OriginAirport_Name
SAN,San Antonio
SFO,San Francisco
BUR,Hollywood Burbank
OAK,Oakland


**Now we want the output as below. So we need to join the two dataframes, but both the dataframe has same column name. So to overcome column name ambuiguity we have to use dataframe.column_name to resolve**

In [0]:
%skip

+------+-----------------+----+--------+
|Origin|OriginAirportName|Dest|Distance|
+------+-----------------+----+--------+
|   OAK|          Oakland| ORD|    1835|
|   SFO|    San Francisco| EWR|    2565|
|   SFO|    San Francisco| BFL|     238|
|   OAK|          Oakland| LAS|     407|
|   SFO|    San Francisco| SCK|      65|
+------+-----------------+----+--------+
only showing top 5 rows

In [0]:
airline_df.join(airport_df, airline_df.Origin == airport_df.Origin, "left")\
          .select(airline_df.Origin,"OriginAirport_Name","Dest","Distance")\
          .where("Origin in ('SAN','SFO','BUR','OAK')")\
          .distinct()\
          .show(5)

+------+------------------+----+--------+
|Origin|OriginAirport_Name|Dest|Distance|
+------+------------------+----+--------+
|   OAK|           Oakland| ORD|    1835|
|   SFO|     San Francisco| EWR|    2565|
|   SFO|     San Francisco| BFL|     238|
|   OAK|           Oakland| LAS|     407|
|   SFO|     San Francisco| SCK|      65|
+------+------------------+----+--------+
only showing top 5 rows


**If we are defining a variable and want to use it inside the select statement then we have to use col function.**

Ex- var1= 2.9
airline_df.select("Origin", expr("Distance * var1 as km_distance"))-------
**This will give an error**

In [0]:
from pyspark.sql.functions import expr
var1= 2.9
airline_df.select("Origin", expr("Distance * var1 as km_distance"))\
          .show(5)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6130927324376266>, line 4
      1 from pyspark.sql.functions import expr
      2 var1= 2.9
      3 airline_df.select("Origin", expr("Distance * var1 as km_distance"))\
----> 4           .show(5)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1123, in DataFrame.show(self, n, truncate, vertical)
   1122 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1123     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:876, in DataFrame._show_string(self, n, truncate, vertical)
    859     except ValueError:
    860         raise PySparkTypeError(
    861             errorClass="NOT_BOOL",
    862             messageParameters={
   (...)
    865             },
  

**So we can simply write it as below**

In [0]:
from pyspark.sql.functions import expr
var1= 2.9
airline_df.select("Origin", (col("Distance") * var1).alias("km_distance") )\
          .show(5)

+------+-----------+
|Origin|km_distance|
+------+-----------+
|   SAN|     1296.3|
|   SAN|     1296.3|
|   SAN|     1296.3|
|   SAN|     1296.3|
|   SAN|     1296.3|
+------+-----------+
only showing top 5 rows


In [0]:
airline_df.write\
    .format("delta")\
    .mode("overwrite")\
    .saveAsTable("airline_data")


In [0]:
%sql
select * from airline_data limit 5

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
1987,10,14,3,741,730,912,849,PS,1451,NA,91,79,NA,23,11,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1987,10,15,4,729,730,903,849,PS,1451,NA,94,79,NA,14,-1,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,NO
1987,10,17,6,741,730,918,849,PS,1451,NA,97,79,NA,29,11,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1987,10,18,7,729,730,847,849,PS,1451,NA,78,79,NA,-2,-1,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,NO,NO
1987,10,19,1,749,730,922,849,PS,1451,NA,93,79,NA,33,19,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES


**Schema Enforcement: Delta does NOT allow schema mismatch by default.**

**Original Distance → IntegerType but i want Distance → StringType**

**mergeSchema does NOT allow data type changes.It only allows adding new columns, not changing existing column types.**

In [0]:
wrong_schema_df = airline_df.withColumn("Distance", airline_df.Distance.cast("string"))


In [0]:
wrong_schema_df.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("airline_data")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-9003849879953944>, line 4
      1 wrong_schema_df.write \
      2     .format("delta") \
      3     .mode("append") \
----> 4     .saveAsTable("airline_data")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1554     req.user_context.user_id = self._user_

In [0]:
wrong_schema_df.write \
    .format("delta") \
    .option("mergeSchema", "true")\
    .mode("append") \
    .saveAsTable("airline_data")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-9003849879953945>, line 5
      1 wrong_schema_df.write \
      2     .format("delta") \
      3     .option("mergeSchema", "true")\
      4     .mode("append") \
----> 5     .saveAsTable("airline_data")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   155

**“Delta Lake enforces schema strictly.mergeSchema supports schema evolution like adding new columns,but it does NOT allow changing existing column data types**

**We will add a new column ingestion_date to the dataframe and it will be successfull as delta support new schema addition**

In [0]:
from pyspark.sql.functions import current_date

new_col_df = airline_df.withColumn("ingestion_date", current_date())

new_col_df.write \
    .format("delta") \
    .option("mergeSchema", "true") \
    .mode("append") \
    .saveAsTable("airline_data")


In [0]:
spark.table("airline_data").printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Ca

In [0]:
%sql
describe extended airline_data

col_name,data_type,comment
Year,int,null
Month,int,null
DayofMonth,int,null
DayOfWeek,int,null
DepTime,string,null
CRSDepTime,int,null
ArrTime,string,null
CRSArrTime,int,null
UniqueCarrier,string,null
FlightNum,int,null
